In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
def week_id_valid(week):
    if week=="w1":    return 1
    elif week=="w2":    return 2
    elif week=="w3": return 3
    else: return 4

In [344]:
# def mape(actual, pred): 
#     actual, pred = np.array(actual), np.array(pred)
#     return np.mean(np.abs((actual - pred) / actual)) * 100

In [345]:
def calculate_underforecast_mape(y_true, y_pred):
    print(len(y_true))
    uf_y_true, uf_y_pred = [], []
    for i in range(len(y_true)):
        if y_true[i] > y_pred[i]:
            uf_y_true.append(y_true)
            uf_y_pred.append(y_pred)
    print(len(uf_y_true))
            
    uf_y_true, uf_y_pred = np.array(uf_y_true), np.array(uf_y_pred)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(uf_y_pred - uf_y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape, axis=0)
            
    #uf_mape = mape(uf_y_true, uf_y_pred)
    return np.average(output_errors)

In [346]:
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error
def calculate_results(y_true, y_pred):
    return {'mae':mean_absolute_error(y_true, y_pred),
            'mape': mean_absolute_percentage_error(y_true, y_pred),
            'uf_mape': calculate_underforecast_mape(y_true, y_pred)}

In [4]:
train_data = pd.read_csv(r"./data/train_data_modified_ml.csv")
train_data

,CategoryCode,ItemCode,Week,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion,WeeklySales,DiscountedAmount
0,category_1,3418,w1,1,NaN,NaN,NaN,0,45,0.0
1,category_1,3418,w2,2,NaN,NaN,NaN,0,50,0.0
2,category_1,3418,w3,3,NaN,NaN,NaN,0,49,0.0
3,category_1,3418,w4,4,NaN,NaN,NaN,0,38,0.0
4,category_1,3418,w1,5,NaN,NaN,NaN,0,53,0.0
...,...,...,...,...,...,...,...,...,...,...
3655,category_2,1105027,w4,16,NaN,NaN,NaN,0,7,0.0
3656,category_2,1105027,w1,17,NaN,NaN,NaN,0,6,0.0
3657,category_2,1105027,w2,18,NaN,NaN,NaN,0,7,0.0
3658,category_2,1105027,w3,19,NaN,NaN,NaN,0,10,0.0


In [5]:
train_data.drop(['DiscountValue', 'DiscountedPrice', 'SellingPrice'], inplace = True, axis = 1)

In [6]:
train_data

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,WeeklySales,DiscountedAmount
0,category_1,3418,w1,1,0,45,0.0
1,category_1,3418,w2,2,0,50,0.0
2,category_1,3418,w3,3,0,49,0.0
3,category_1,3418,w4,4,0,38,0.0
4,category_1,3418,w1,5,0,53,0.0
...,...,...,...,...,...,...,...
3655,category_2,1105027,w4,16,0,7,0.0
3656,category_2,1105027,w1,17,0,6,0.0
3657,category_2,1105027,w2,18,0,7,0.0
3658,category_2,1105027,w3,19,0,10,0.0


In [7]:
valid_data = pd.read_csv(r'./data/validation_data_modified_new.csv')
valid_data['id'] = [week_id_valid(week)+20 for week in valid_data['Week']]
valid_data.sort_values(by='id', inplace = True)
valid_data

,CategoryCode,ItemCode,Week,OnPromo,WeeklySales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion,DiscountedAmount,id
0,category_2,1006090,w1,0,9,21,NaN,NaN,NaN,0,0.00,21
81,category_1,1085749,w1,1,17,21,0.10,63.00,70.0,1,7.00,21
170,category_1,1090105,w1,0,9,21,NaN,NaN,NaN,0,0.00,21
312,category_2,836125,w1,0,34,21,NaN,NaN,NaN,0,0.00,21
255,category_1,1090294,w1,0,24,21,NaN,NaN,NaN,0,0.00,21
...,...,...,...,...,...,...,...,...,...,...,...,...
140,category_3,1066570,w4,1,41,24,0.15,38.25,45.0,1,6.75,24
272,category_2,1090276,w4,0,8,24,NaN,NaN,NaN,0,0.00,24
271,category_1,36898,w4,0,142,24,NaN,NaN,NaN,0,0.00,24
120,category_1,1050046,w4,0,62,24,NaN,NaN,NaN,0,0.00,24


In [8]:
valid_data.drop(['OnPromo', 'DiscountValue', 'DiscountedPrice', 'SellingPrice', 'id'], axis = 1, inplace = True)

In [9]:
valid_data

,CategoryCode,ItemCode,Week,WeeklySales,WeekID,OnPromotion,DiscountedAmount
0,category_2,1006090,w1,9,21,0,0.00
81,category_1,1085749,w1,17,21,1,7.00
170,category_1,1090105,w1,9,21,0,0.00
312,category_2,836125,w1,34,21,0,0.00
255,category_1,1090294,w1,24,21,0,0.00
...,...,...,...,...,...,...,...
140,category_3,1066570,w4,41,24,1,6.75
272,category_2,1090276,w4,8,24,0,0.00
271,category_1,36898,w4,142,24,0,0.00
120,category_1,1050046,w4,62,24,0,0.00


In [10]:
test_data = pd.read_csv('./data/test_data_modified_new.csv')
test_data['id'] = [week_id_valid(week)+20 for week in test_data['Week']]
test_data

,CategoryCode,ItemCode,Week,OnPromo,PredictedSales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion,DiscountedAmount,id
0,category_2,32245,w1,0,NaN,21,NaN,NaN,NaN,0,0.0,21
1,category_1,1056463,w1,0,NaN,21,NaN,NaN,NaN,0,0.0,21
2,category_1,119554,w1,0,NaN,21,NaN,NaN,NaN,0,0.0,21
3,category_2,815101,w1,0,NaN,21,NaN,NaN,NaN,0,0.0,21
4,category_1,1054978,w1,0,NaN,21,NaN,NaN,NaN,0,0.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...
372,category_1,145330,w4,0,NaN,24,NaN,NaN,NaN,0,0.0,24
373,category_1,1032568,w4,0,NaN,24,NaN,NaN,NaN,0,0.0,24
374,category_2,838456,w4,0,NaN,24,NaN,NaN,NaN,0,0.0,24
375,category_4,1003192,w4,0,NaN,24,NaN,NaN,NaN,0,0.0,24


In [11]:
test_data.drop(['OnPromo', 'DiscountValue', 'DiscountedPrice', 'SellingPrice', 'id'], axis = 1, inplace = True)

In [12]:
test_data

,CategoryCode,ItemCode,Week,PredictedSales,WeekID,OnPromotion,DiscountedAmount
0,category_2,32245,w1,NaN,21,0,0.0
1,category_1,1056463,w1,NaN,21,0,0.0
2,category_1,119554,w1,NaN,21,0,0.0
3,category_2,815101,w1,NaN,21,0,0.0
4,category_1,1054978,w1,NaN,21,0,0.0
...,...,...,...,...,...,...,...
372,category_1,145330,w4,NaN,24,0,0.0
373,category_1,1032568,w4,NaN,24,0,0.0
374,category_2,838456,w4,NaN,24,0,0.0
375,category_4,1003192,w4,NaN,24,0,0.0


In [13]:
train_data_items = train_data['ItemCode'].values

In [14]:
valid_data_bel = valid_data[~valid_data['ItemCode'].isin(train_data_items)]

In [15]:
test_data_bel = test_data[~test_data['ItemCode'].isin(train_data_items)]

In [16]:
valid_data = valid_data[valid_data['ItemCode'].isin(train_data_items)]

In [17]:
test_data = test_data[test_data['ItemCode'].isin(train_data_items)]

In [18]:
test_data_bel.to_csv('test_data_bel_thres.csv')

In [19]:
valid_data_bel.to_csv('valid_data_bel_thres.csv')

In [20]:
test_data.to_csv('test_data_abv_thres.csv')

In [21]:
valid_data.to_csv('valid_data_abv_thres.csv')

In [22]:
train_valid = pd.concat([train_data, valid_data])
train_valid

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,WeeklySales,DiscountedAmount
0,category_1,3418,w1,1,0,45,0.00
1,category_1,3418,w2,2,0,50,0.00
2,category_1,3418,w3,3,0,49,0.00
3,category_1,3418,w4,4,0,38,0.00
4,category_1,3418,w1,5,0,53,0.00
...,...,...,...,...,...,...,...
140,category_3,1066570,w4,24,1,41,6.75
272,category_2,1090276,w4,24,0,8,0.00
271,category_1,36898,w4,24,0,142,0.00
120,category_1,1050046,w4,24,0,62,0.00


In [23]:
train_valid.sort_values(by = ['ItemCode', 'WeekID'], inplace = True, ignore_index= True)
train_valid

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,WeeklySales,DiscountedAmount
0,category_1,3418,w1,1,0,45,0.0
1,category_1,3418,w2,2,0,50,0.0
2,category_1,3418,w3,3,0,49,0.0
3,category_1,3418,w4,4,0,38,0.0
4,category_1,3418,w1,5,0,53,0.0
...,...,...,...,...,...,...,...
4000,category_2,1105027,w4,20,0,13,0.0
4001,category_2,1105027,w1,21,0,16,0.0
4002,category_2,1105027,w2,22,0,15,0.0
4003,category_2,1105027,w3,23,0,9,0.0


## Feature Engineering

In [282]:
df = train_valid.copy()
train_test = train_data.copy()
test_df = test_data.copy()

#test_df.columns = ['CategoryCode','ItemCode','Week','WeekID', 'OnPromotion', 'WeeklySales', 'DiscountedAmount']
test_df
le = LabelEncoder()
le.fit(df['ItemCode'])

df['ItemCode_Label'] = le.transform(df['ItemCode'])
train_test['ItemCode_Label'] = le.transform(train_test['ItemCode'])
test_df['ItemCode_Label'] = le.transform(test_df['ItemCode'])

In [283]:
test_df

,CategoryCode,ItemCode,Week,PredictedSales,WeekID,OnPromotion,DiscountedAmount,ItemCode_Label
0,category_2,32245,w1,NaN,21,0,0.0,12
1,category_1,1056463,w1,NaN,21,0,0.0,133
2,category_1,119554,w1,NaN,21,0,0.0,45
3,category_2,815101,w1,NaN,21,0,0.0,84
4,category_1,1054978,w1,NaN,21,0,0.0,132
...,...,...,...,...,...,...,...,...
372,category_1,145330,w4,NaN,24,0,0.0,55
373,category_1,1032568,w4,NaN,24,0,0.0,120
374,category_2,838456,w4,NaN,24,0,0.0,89
375,category_4,1003192,w4,NaN,24,0,0.0,105


In [284]:
df

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,WeeklySales,DiscountedAmount,ItemCode_Label
0,category_1,3418,w1,1,0,45,0.0,0
1,category_1,3418,w2,2,0,50,0.0,0
2,category_1,3418,w3,3,0,49,0.0,0
3,category_1,3418,w4,4,0,38,0.0,0
4,category_1,3418,w1,5,0,53,0.0,0
...,...,...,...,...,...,...,...,...
4000,category_2,1105027,w4,20,0,13,0.0,182
4001,category_2,1105027,w1,21,0,16,0.0,182
4002,category_2,1105027,w2,22,0,15,0.0,182
4003,category_2,1105027,w3,23,0,9,0.0,182


In [285]:
def add_features(dataframe, shift_val = 4):
    dataframe['Last-3_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val)
    dataframe['Last-3_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 1)
    dataframe['Last-4_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 1)
    dataframe['Last-4_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 1) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 2)
    dataframe['Last-5_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 2)
    dataframe['Last-5_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 2) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val + 3)
#     dataframe['Next_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val)
#     dataframe['Next_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 1)
#     dataframe['Next+1_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 1)
#     dataframe['Next+1_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 1) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 2)
#     dataframe['Next+2_Week_Sales'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 2)
#     dataframe['Next+2_Week_Diff'] = dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 2) - dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val - 3)
#     dataframe['Average_Last_3_weeks'] = (dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val) + dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val+1) + dataframe.groupby(['ItemCode'])['WeeklySales'].shift(shift_val+2)) / 3
#     dataframe['Average_Next_3_weeks'] = (dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val) + dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val-1) + dataframe.groupby(['ItemCode'])['WeeklySales'].shift(-shift_val-2)) / 3

    
    
    return dataframe

In [286]:
def drop_columns(dataframe, columns_to_drop):
    for column in columns_to_drop:
        if column in dataframe.columns:
            dataframe = dataframe.drop([column], axis = 1)
            
    return dataframe

In [287]:
df = add_features(df)
df = df.dropna()
df.head()

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,WeeklySales,DiscountedAmount,ItemCode_Label,Last-3_Week_Sales,Last-3_Week_Diff,Last-4_Week_Sales,Last-4_Week_Diff,Last-5_Week_Sales,Last-5_Week_Diff
7,category_1,3418,w4,8,0,47,0.0,0,38.0,-11.0,49.0,-1.0,50.0,5.0
8,category_1,3418,w1,9,0,58,0.0,0,53.0,15.0,38.0,-11.0,49.0,-1.0
9,category_1,3418,w2,10,0,88,0.0,0,38.0,-15.0,53.0,15.0,38.0,-11.0
10,category_1,3418,w3,11,0,98,0.0,0,50.0,12.0,38.0,-15.0,53.0,15.0
11,category_1,3418,w4,12,0,61,0.0,0,47.0,-3.0,50.0,12.0,38.0,-15.0


In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2724 entries, 7 to 4004
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CategoryCode       2724 non-null   object 
 1   ItemCode           2724 non-null   int64  
 2   Week               2724 non-null   object 
 3   WeekID             2724 non-null   int64  
 4   OnPromotion        2724 non-null   int64  
 5   WeeklySales        2724 non-null   int64  
 6   DiscountedAmount   2724 non-null   float64
 7   ItemCode_Label     2724 non-null   int64  
 8   Last-3_Week_Sales  2724 non-null   float64
 9   Last-3_Week_Diff   2724 non-null   float64
 10  Last-4_Week_Sales  2724 non-null   float64
 11  Last-4_Week_Diff   2724 non-null   float64
 12  Last-5_Week_Sales  2724 non-null   float64
 13  Last-5_Week_Diff   2724 non-null   float64
dtypes: float64(7), int64(5), object(2)
memory usage: 319.2+ KB


In [289]:
columns_to_drop = ['WeeklySales', 'ItemCode']
cat_col = ['CategoryCode', 'Week']
quan_col = ['DiscountedAmount', 'Last-3_Week_Sales', 'Last-3_Week_Diff', 'Last-4_Week_Sales', 'Last-4_Week_Diff', 'Last-5_Week_Sales', 'Last-5_Week_Diff']#, 'Average_Last_3_weeks']

In [347]:
def run_model(model, dataframe, target_week, pipeline = None, mode = 'train', train_dataframe = None, test_dataframe = None, buffer = 0):
    
    if mode == 'train':
    
        train = dataframe[dataframe['WeekID'] < target_week]
        val = dataframe[dataframe['WeekID'] == target_week]
        
        #print(train['WeekID'].value_counts())

        xtr, xts = drop_columns(train, columns_to_drop), drop_columns(val, columns_to_drop)
        ytr, yts = train['WeeklySales'].values, val['WeeklySales'].values
        xtr = xtr.reindex(sorted(xtr.columns), axis=1)
        xts = xts.reindex(sorted(xts.columns), axis=1)

        quan_pipeline = Pipeline([('std_scaler', StandardScaler())])
        quan_transformed = quan_pipeline.fit_transform(xtr[quan_col])

        data_pipeline = ColumnTransformer([('numerical', quan_pipeline, quan_col),
                                           ('categorical', OneHotEncoder(handle_unknown = 'ignore'), cat_col),])
        
        xtr_sc = data_pipeline.fit_transform(xtr)
        xts_sc = data_pipeline.transform(xts)

        model.fit(xtr_sc, np.log1p(ytr))
        p = np.expm1(model.predict(xts_sc)) + buffer # on validation

        error = calculate_results(yts, p)
        print(f'Week {target_week} - Error {error}')
        #mean_error.append(error)
        
        return model, data_pipeline
        
    elif mode == 'inference':
        c = []
    
        train_test = pd.concat([train_dataframe, test_dataframe[test_dataframe['WeekID']==target_week]])
        train_test = add_features(train_test)

        train = train_test[train_test['WeekID'] < target_week]
        train = train.dropna()

        test = train_test[train_test['WeekID']==target_week]

        xtest = drop_columns(test, columns_to_drop)
        ytest = test['WeeklySales'].values

        xtest = xtest.reindex(sorted(xtest.columns), axis=1)
        print(xtest.info())
        xtest_sc = pipeline.transform(xtest.drop(['PredictedSales'], axis = 1))

        test_p = np.expm1(model.predict(xtest_sc)) # test data
        #train_test[train_test['id']==week]['WeeklySales'] = test_p
        train_test.loc[train_test["WeekID"] == target_week, 'WeeklySales'] = test_p
        #train_test.drop(['AverageSales'], axis = 1, inplace = True)

        c.append({'CategoryCode': test['CategoryCode'],'ItemCode':test['ItemCode'],
                  'Week':test['Week'],'WeekID':test['WeekID'], 'OnPromotion': test['OnPromotion'], 'PredictedSales':test_p})
        return c

In [348]:
model_21, pipeline_21 = run_model(model=XGBRegressor(), dataframe=df, target_week=21)
model_22, pipeline_22 = run_model(model=XGBRegressor(), dataframe=df, target_week=22)
model_23, pipeline_23 = run_model(model=XGBRegressor(), dataframe=df, target_week=23)
model_24, pipeline_24 = run_model(model=XGBRegressor(), dataframe=df, target_week=24)

85
50
Week 21 - Error {'mae': 26.179448005732368, 'mape': 1.0100614536073453, 'uf_mape': 1.0100614536073453}
88
55
Week 22 - Error {'mae': 30.84092031012882, 'mape': 0.6153632582484735, 'uf_mape': 0.6153632582484736}
89
64
Week 23 - Error {'mae': 28.59394937552763, 'mape': 0.4407462456273596, 'uf_mape': 0.44074624562735953}
83
52
Week 24 - Error {'mae': 27.846137267997467, 'mape': 0.7211462348164698, 'uf_mape': 0.7211462348164698}


In [331]:
test_21 = run_model(model = model_21, dataframe = None, target_week=21, pipeline= pipeline_21, mode = 'inference', train_dataframe=train_data, test_dataframe=test_df )
test_22 = run_model(model = model_22, dataframe = None, target_week=22, pipeline= pipeline_22, mode = 'inference', train_dataframe=train_data, test_dataframe=test_df )
test_23 = run_model(model = model_23, dataframe = None, target_week=23, pipeline= pipeline_23, mode = 'inference', train_dataframe=train_data, test_dataframe=test_df )
test_24 = run_model(model = model_24, dataframe = None, target_week=24, pipeline= pipeline_24, mode = 'inference', train_dataframe=train_data, test_dataframe=test_df )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 92
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CategoryCode       86 non-null     object 
 1   DiscountedAmount   86 non-null     float64
 2   ItemCode_Label     86 non-null     float64
 3   Last-3_Week_Diff   86 non-null     float64
 4   Last-3_Week_Sales  86 non-null     float64
 5   Last-4_Week_Diff   86 non-null     float64
 6   Last-4_Week_Sales  86 non-null     float64
 7   Last-5_Week_Diff   86 non-null     float64
 8   Last-5_Week_Sales  86 non-null     float64
 9   OnPromotion        86 non-null     int64  
 10  PredictedSales     0 non-null      float64
 11  Week               86 non-null     object 
 12  WeekID             86 non-null     int64  
dtypes: float64(9), int64(2), object(2)
memory usage: 9.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 93 to 187
Data columns (total 13 columns):
 #   

In [276]:
pd.DataFrame(test_24[0])

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,PredictedSales
282,category_3,893824,w4,24,0,33.009186
283,category_1,371239,w4,24,0,11.944530
284,category_1,35449,w4,24,0,13.436604
285,category_3,1081078,w4,24,0,13.614229
286,category_1,416212,w4,24,0,12.976562
...,...,...,...,...,...,...
372,category_1,145330,w4,24,0,17.454487
373,category_1,1032568,w4,24,0,8.044353
374,category_2,838456,w4,24,0,25.340576
375,category_4,1003192,w4,24,0,42.319683


In [277]:
submissions = test_data.drop(['PredictedSales'], axis = 1)
submissions

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,DiscountedAmount
0,category_2,32245,w1,21,0,0.0
1,category_1,1056463,w1,21,0,0.0
2,category_1,119554,w1,21,0,0.0
3,category_2,815101,w1,21,0,0.0
4,category_1,1054978,w1,21,0,0.0
...,...,...,...,...,...,...
372,category_1,145330,w4,24,0,0.0
373,category_1,1032568,w4,24,0,0.0
374,category_2,838456,w4,24,0,0.0
375,category_4,1003192,w4,24,0,0.0


In [278]:
all_concat = pd.concat([pd.DataFrame(test_21[0]), pd.DataFrame(test_22[0]), pd.DataFrame(test_23[0]), pd.DataFrame(test_24[0])])
all_concat

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,PredictedSales
0,category_2,32245,w1,21,0,45.793415
1,category_1,1056463,w1,21,0,78.230904
2,category_1,119554,w1,21,0,147.668991
3,category_2,815101,w1,21,0,22.506603
4,category_1,1054978,w1,21,0,73.474388
...,...,...,...,...,...,...
372,category_1,145330,w4,24,0,17.454487
373,category_1,1032568,w4,24,0,8.044353
374,category_2,838456,w4,24,0,25.340576
375,category_4,1003192,w4,24,0,42.319683


In [279]:
submissions1 = pd.merge(submissions,all_concat, on=["CategoryCode","ItemCode","Week","WeekID", "OnPromotion"], how = "left")
submissions1

,CategoryCode,ItemCode,Week,WeekID,OnPromotion,DiscountedAmount,PredictedSales
0,category_2,32245,w1,21,0,0.0,45.793415
1,category_1,1056463,w1,21,0,0.0,78.230904
2,category_1,119554,w1,21,0,0.0,147.668991
3,category_2,815101,w1,21,0,0.0,22.506603
4,category_1,1054978,w1,21,0,0.0,73.474388
...,...,...,...,...,...,...,...
345,category_1,145330,w4,24,0,0.0,17.454487
346,category_1,1032568,w4,24,0,0.0,8.044353
347,category_2,838456,w4,24,0,0.0,25.340576
348,category_4,1003192,w4,24,0,0.0,42.319683
